In [1]:
# Importing basic Libraries for EDA

import pandas as pd
import regex as re
from ast import literal_eval
import warnings

warnings.filterwarnings("ignore")


pd.set_option("display.max_columns", 10)

# Create a list of stopwords
import nltk
import spacy

stopwords_nltk = nltk.corpus.stopwords.words("english")
nlp = spacy.load("en_core_web_lg")
stopwords_spacy = nlp.Defaults.stop_words
stopwords = list(
    set(
        stopwords_nltk
        + list(stopwords_spacy)
        + list(stopwords_nltk)
        + list("abdefghijklmnopqstuvwxyz")
    )
)
print(len(stopwords))

398


In [2]:

df = pd.read_csv("Data/combined_df.csv")
df.head()

,Head,Body,Tags
0,Brain Segmentation To 3D Model,\r\nMy goal is to take a dataset of Brain Tumo...,"['computer-vision', 'python']"
1,Active Learning regression with Random Forest,\r\nI have a dataset of about 8k points and I ...,"['machine-learning', 'regression', 'uncertaint..."
2,Comparing Reinforcement Learning models,\r\nI am currently completing my thesis on opt...,"['reinforcement-learning', 'policy-gradients',..."
3,"Why ""Good Model"" that performs great on holdou...",\r\nI have this binary regression model that h...,"['deep-learning', 'deep-neural-networks', 'pre..."
4,What are Reservoir computers used for today?,\r\nReservoir computers were very popular in t...,"['machine-learning', 'recurrent-neural-network..."


In [3]:

# Dropping the rows with missing values in the body and tags columns and dropping the duplicates.
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [4]:
df["Text"] = df["Head"] + " " + df["Body"]

# Cleaning the tags
df["Tags Count"] = df["Tags"].apply(lambda x: len(literal_eval(x)))
df["Tags Count"] = df['Tags Count'].astype("int16")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48803 entries, 0 to 48810
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Head        48803 non-null  object
 1   Body        48803 non-null  object
 2   Tags        48803 non-null  object
 3   Text        48803 non-null  object
 4   Tags Count  48803 non-null  int16 
dtypes: int16(1), object(4)
memory usage: 2.0+ MB


In [7]:

# Removing the rows with more than 5 tags
df = df[df["Tags Count"] <= 5]

In [8]:
def clean_text(text):
    text = re.sub(r"[^a-zA-Z]", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = text.lower()
    text = text.strip()
    # Lemmatization and removing stopwords
    text = " ".join(
        [
            token.lemma_
            for token in nlp(text)
            if token.lemma_ not in stopwords and token.lemma_ != "-PRON-"
        ]
    )
    return text

df['Text'] = df['Text'].apply(clean_text)

In [9]:
df.head()

,Head,Body,Tags,Text,Tags Count
0,Brain Segmentation To 3D Model,\r\nMy goal is to take a dataset of Brain Tumo...,"['computer-vision', 'python']",brain segmentation model goal dataset brain tu...,2
1,Active Learning regression with Random Forest,\r\nI have a dataset of about 8k points and I ...,"['machine-learning', 'regression', 'uncertaint...",active learning regression random forest I dat...,5
2,Comparing Reinforcement Learning models,\r\nI am currently completing my thesis on opt...,"['reinforcement-learning', 'policy-gradients',...",compare reinforcement learning model I current...,4
3,"Why ""Good Model"" that performs great on holdou...",\r\nI have this binary regression model that h...,"['deep-learning', 'deep-neural-networks', 'pre...",good model perform great holdout validation da...,5
4,What are Reservoir computers used for today?,\r\nReservoir computers were very popular in t...,"['machine-learning', 'recurrent-neural-network...",reservoir computer use today reservoir compute...,3


In [10]:
df.to_csv("Data/cleaned_data.csv", index=False)

## Now our data is in a good shape, we can start to do some analysis on it.